In [2]:
import pandas as pd

# ------------------------------
# Paso 1: Cargar bases de hogares
# ------------------------------
hogares_2005 = pd.read_csv(
    r"C:\Users\Machi Pardini\Documents\GitHub\Big-Data-UBA--Grupo-3\TP02\hogares_2005.csv"
)
hogares_2025 = pd.read_excel(
    r"C:\Users\Machi Pardini\Documents\GitHub\Big-Data-UBA--Grupo-3\TP02\usu_hogar_T125.xlsx"
)

# ------------------------------
# Paso 2: Limpiar nombres de columnas
# ------------------------------
def limpiar_columnas(df):
    df.columns = (
        df.columns.str.strip()
                  .str.lower()
                  .str.replace(' ', '_')
                  .str.replace('-', '_')
    )
    return df

hogares_2005 = limpiar_columnas(hogares_2005)
hogares_2025 = limpiar_columnas(hogares_2025)

# ------------------------------
# Paso 3: Eliminar filas completamente vacías
# ------------------------------
hogares_2005 = hogares_2005.dropna(how='all')
hogares_2025 = hogares_2025.dropna(how='all')

# ------------------------------
# Paso 4: Crear variable miembros_hogar si no existe
# ------------------------------
if 'miembros_hogar' not in hogares_2005.columns:
    hogares_2005['miembros_hogar'] = hogares_2005.groupby('nro_hogar')['codusu'].transform('count')

if 'miembros_hogar' not in hogares_2025.columns:
    hogares_2025['miembros_hogar'] = hogares_2025.groupby('nro_hogar')['codusu'].transform('count')

# ------------------------------
# Paso 5: Concatenar hogares 2005 y 2025
# ------------------------------
hogares_final = pd.concat([hogares_2005, hogares_2025], ignore_index=True)

# ------------------------------
# Paso 6: Cargar respondieron_completa
# ------------------------------
respondieron_completa = pd.read_csv(
    r"C:\Users\Machi Pardini\Documents\GitHub\Big-Data-UBA--Grupo-3\TP02\respondieron_completa.csv",
    low_memory=False
)

# ------------------------------
# Paso 7: Merge seguro para mantener todos los datos de respondieron_completa
# ------------------------------
hogares_final_unida = respondieron_completa.merge(
    hogares_final,
    on='codusu',
    how='left',  # conservar todos los registros de respondieron_completa
    suffixes=('', '_hogar')
)

# ------------------------------
# Paso 8: Revisar columnas duplicadas
# ------------------------------
cols_final = []
for c in hogares_final_unida.columns:
    if c.endswith('_hogar'):
        cols_final.append(c)
    else:
        cols_final.insert(0, c)

hogares_final_unida = hogares_final_unida[cols_final]

# ------------------------------
# Paso 9: Verificar base final
# ------------------------------
print("Filas:", hogares_final_unida.shape[0])
print("Columnas:", hogares_final_unida.shape[1])
print("\nConteo por año:")
print(hogares_final_unida['ano4'].value_counts(dropna=False))
print("\nConteo de pobres:")
print(hogares_final_unida['pobre'].value_counts(dropna=False))


Filas: 23838
Columnas: 357

Conteo por año:
ano4
2005.0    14481
2025.0     9357
Name: count, dtype: int64

Conteo de pobres:
pobre
0    20928
1     2910
Name: count, dtype: int64


In [3]:
hogares_final_unida.to_csv(r"C:\Users\Machi Pardini\Documents\GitHub\Big-Data-UBA--Grupo-3\TP02\hogares_final_unida.csv", 
                           index=False, encoding="utf-8")
